## Bài toán
 - Dự đoán khả năng tiến triển của bệnh tiểu đường thông qua các chỉ số sinh lý của cơ thể.


## Thông tin dữ liệu:

- Số lượng mẫu: 442 (thông tin từ 442 bệnh nhân)
- Thông tin ccs thuộc tính (10 cột đầu tiên):
  - Age(tuổi)
  - Sex (giới tính) 
  - Body mass index (chỉ số khối cơ thể)
  - Average blood pressure(huyết ap trung bình)
  - S1, S2, S3, S4, S5, S6 (sáu phép đo huyết thanh khác)
- Mục tiêu: Cột 11, chỉ số đánh giá mức độ tiến triển của bệnh sau 1 năm điều trị

**! Chú ý: Các thuộc tính trong dữ liệu đã được chuẩn hóa**

Xem thêm thông tin về nguồn dữ liệu tại: (https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html)


## Hướng giải quyết

Giả sử rằng khả năng tiến triển của bệnh tiểu đường (ký hiệu: `y`) là đại lượng phụ thuộc tuyến tính vào các thông tin sinh lý của bệnh nhân như các thuộc tính đã mô tả ở trên (tuổi, giới tính, chỉ số khối, ... - ký hiệu: $x_1, x_2, .. x_n$) : 

$y = w_0 + w_1*x_1 + w_1*x_2 + ... + w_n*x_n$

Mục tiêu: Tìm được bộ trọng số $[w_0, w_1, w_2... w_n]$ biểu diễn mối quan hệ này. 

## Thư viện sử dụng 

- matplotlib: phục vụ vẽ các đồ thị 
- numpy: tính toán các phép biến đổi trên ma trận / vector 
- math: thực hiện một số hàm tính toán đơn giản
- pandas: Xử lý dữ liệu bảng hai chiều 
- scikit-learn: (sklearn) thư viện hỗ trợ xây dựng các mô hình học máy, các hàm huấn luyện và đánh giá 


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import math 

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score


## 1. Đọc dữ liệu 

Dữ liệu về bệnh tiểu đường được hỗ trợ bởi sklearn, đọc dữ liệu thông qua hàm `datasets.load_diabetes()` 

Xem thêm các bộ dữ liệu khác tại https://scikit-learn.org/stable/datasets/index.html#toy-datasets. 

Dữ liệu nhận về ở dạng object với các thành phần thuộc tính: 
- data: ma trận 2 chiều (442x10) - các thông tin bệnh nhân được chuẩn hoá về dạng số thực
- target: mảng các số thực (442,) - chỉ số tiến triển của bệnh tiểu đường

In [ ]:
# sử dụng hàm load_diabetes để load dữ liệu
diabetes = datasets.load_diabetes()
print("Số chiều input: ", diabetes.data.shape)
print("Số chiều target y tương ứng: ", diabetes.target.shape)
print()

print("5 mẫu dữ liệu đầu tiên:")
print("input: ", diabetes.data[:5])
print("target: ",diabetes.target[:5])

### Chia dữ liệu làm 2 phần: huấn luyện trên 362 mẫu và đánh giá trên 80 mẫu

In [ ]:
diabetes_X = diabetes.data

diabetes_X_train = diabetes_X[:-80]
diabetes_y_train = diabetes.target[:-80]

diabetes_X_test = diabetes_X[-80:]
diabetes_y_test = diabetes.target[-80:]


## 2. Xây dựng mô hình Regression sử dụng Sklearn

Thử nghiệm xây dựng và huấn luyện mô hình hồi quy (Linear Regression)
- [Linear Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) `linear_model.LinearRegression()`

In [ ]:
# Xây dựng mô hình sử dụng thư viện scikit-learn
regr = linear_model.LinearRegression()


## 3. Training mô hình

Sử dụng dữ liệu đã được chia ở bước trước đó để thực hiện huấn luyện mô hình 

=> Tìm được bộ trọng số $[w_0, w_1, ... w_n]$

In [ ]:
# huấn luyện mô hình
regr.fit(diabetes_X_train, diabetes_y_train)
print("[w1, ... w_n] = ", regr.coef_)
print("w0 = ", regr.intercept_)

## 4. Dự đoán các mẫu dữ liệu trong tập test 

In [ ]:
# Thực hiện suy diễn sau khi huấn luyện
diabetes_y_pred = regr.predict(diabetes_X_test)
pd.DataFrame(data=np.array([diabetes_y_test, diabetes_y_pred,
                            abs(diabetes_y_test - diabetes_y_pred)]).T,
             columns=["Thực tế", "Dự đoán", "Lệch"])  

## 5. Đánh giá 

Sử dụng độ đo RMSE tính căn bậc 2 của trung bình bình phương lỗi. 
> $\text{RMSE}(y, \hat{y}) = \sqrt{\frac{1}{n_\text{samples}} \sum_{i=0}^{n_\text{samples} - 1} (y_i - \hat{y}_i)^2}.$

- Lỗi càng nhỏ càng thể hiện mô hình có khả năng học và dự đoán hiệu quả

In [ ]:
math.sqrt(mean_squared_error(diabetes_y_test, diabetes_y_pred))

In [ ]:
import seaborn as sns
sns.distplot(diabetes_y_test, )
pd.DataFrame(data=diabetes_y_test, columns=["values"]).describe()